### Pricing d'un Call asiatique - Variable de Controle de Kemna-Vorst (Sans Dividendes)

In [1]:
#Importation des librairies que j'utilise dans le Notebook
import numpy as np
import random as rd
from math import sqrt , log , exp 
import scipy.stats as sps

In [2]:
#parametres du probleme
sigma = 1 #volatilité du sous-jacent
T = 1 #maturite de l'option
s = 10 #prix initiale du sous jacent
r = 0.05 #taux d'interet sans risque , supposé constant :)
K = 5 # Strike du call asiatique

In [3]:
# Simulatation d'une discretisation sur un horizon fini de la trajectoire d'un MB Par Decomposition de Cholesky
def Brownian_simulator(t): # t vecteur des instant ti ou on veut simuler notre trajectoire bronienne
    n=len(t)
    L = np.tri(n, n, k=0)  
    D =np.eye(n)
    D[0,0] = np.sqrt(t[0])
    for i in range(1,n):
        D[i,i] =  np.sqrt(t[i] - t[i-1])


    T =L@D # T est la matrice de la decomposition de cholesky de la cov de W_t1,..tn

    #Simulation d'un n-vecteur gaussien de matrice de variance-covariance = I_n
    def Box_muller():
        U1, U2 = rd.random(), rd.random()  
        Z0 = np.sqrt(-2 * np.log(U1)) * np.cos(2 * np.pi * U2)  
        Z1 = np.sqrt(-2 * np.log(U1)) * np.sin(2 * np.pi * U2)  
        return Z0, Z1

    Z = []
    d = n + n%2
    for i in range(d//2):
        a,b = Box_muller()
        Z.append(a)
        Z.append(b)
    if n%2 == 1:
        Z.pop()
    Z = np.array(Z)

    return T@Z # Resultat :)



In [4]:
#Prix Black_scholes d'un Call , pas de dividendes

def Call_BS(s,K,r ,sigma , to ):
    normale = sps.norm()
    d1 = (log(s/K) + (r+ sigma**2 / 2)*to ) / (sigma * sqrt(to))
    d2 = d1 - sigma * sqrt(to)
    N1 = normale.cdf(d1)
    N2 = normale.cdf(d2)
    return s*N1 - K*exp(-r*to)*N2


In [12]:
def Kemna_Vorst() : #retourne la realisation du payoff de l'asiatique et de la variable de controle de KV 
    T = 1
    n= 50 #nombre d'elements pour une seule realisation - 1/n est le pas d'integration 
    time = np.array([(2*k-1)*T/(2*n) for k in range(1,n+1)]) #mon vecteur temps (2*k-1)T/(2*n) 
    W = Brownian_simulator(time) # (WT/2n , ... W(2k-1)/2n)
    f1 = s*np.exp((r - sigma**2/2)*time + sigma*W)
    f2 = W.copy()
    int1 , int2 = np.sum(f1)/n , np.sum(f2)/n
    S2 = s* exp( -(r + sigma**2 /6 ) * T / 2 ) * exp( (r - sigma**2 / 6) * T + sigma * int2 / T )
    X = max(int1 - K , 0)
    Kv = max(S2 - K , 0)
    return X , Kv

N = 2**10 # Number of Simulations
Naive , Naive2 , Kemna , Kemna2 = 0 , 0 , 0 , 0

for _ in range(N):
    X , Kv = Kemna_Vorst()
    Kemna += X - Kv 
    Kemna2 += (X - Kv)**2
    Naive += X
    Naive2 += X**2


Kemna *= exp(-r*T)/N
Naive *= exp(-r*T)/N
Kemna2 *= exp(-2*r*T)/N 
Naive2 *= exp(-2*r*T)/N
Naive2 -= Naive**2
Kemna2 -= Kemna**2

Kemna += Call_BS(s* exp( -(r + sigma**2 /6 ) * T / 2 )   ,K,r,sigma / sqrt(3), to = T)

print(f"pour S_0 ={s}  K = {K} , sigma ={sigma} , r = {r}  , T= {T} ")

print(f"Estimateur Naif :  Prix {Naive} Ecartype {sqrt(Naive2)}")
print(f"Estimateur Kemna-Vorst :  Prix {Kemna} Ecartype {sqrt(Kemna2)}")

#L'ecart type me parrait enorme , meme si les valeurs sont proches ... 
    

pour S_0 =10  K = 5 , sigma =1 , r = 0.05 = T= 1 
Estimateur Naif :  Prix 4.8841191158238555 Ecartype 7.036162423144364
Estimateur Kemna-Vorst :  Prix 5.13897736369743 Ecartype 1.9363535377097454
